# Configuração, Inicialização e Importações

In [17]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução.

In [18]:
import pyspark # Importa a biblioteca principal do PySpark
from pyspark.sql import SparkSession # mporta a classe SparkSession, usada para iniciar a sessão Spark.
from pyspark.ml.regression import LinearRegression # Importa a classe LinearRegression do MLlib, o modelo de regressão linear
from pyspark.ml.feature import VectorAssembler, MinMaxScaler # Importa VectorAssembler (para agrupar features em um vetor) e MinMaxScaler (para normalização)

In [19]:
from google.colab import drive # Importa o módulo drive do Google Colab.
drive.mount('/content/drive') # Monta o Google Drive no ambiente.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [21]:
pagamentos_pedido_preprocessados = spark.read.option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/output/pagamento_pedido_preprocessados_parquet') #Carrega dados que já passaram por normalização de outras etapas.
join_itens_pedido = spark.read.option('header','true').parquet('/content/drive/MyDrive/Material de apoio - M27/join_itens_pedido.parquet') # Carrega o DataFrame principal que contém as features do produto e o rótulo (valor_frete)

In [22]:
pagamentos_pedido_preprocessados.show(n=5, truncate=False) # Exibe as 5 primeiras linhas do DataFrame de pagamentos para verificação
join_itens_pedido.show(n=5, truncate=False) # xibe as 5 primeiras linhas do DataFrame de itens de pedido para verificação

+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|id_pedido                       |sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamento_normalizado|
+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                  |credit_card   |8                 |99.33          |0.007269424652080491       |0.3333333333333333            |
|a9810da82917af2d9aefd1278f1dcfa0|1                  |credit_card   |1                 |24.39          |0.0017849719849415402      |0.041666666666666664          |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                  |credit_card   |1                 |65.71          |0.004808958963940492       |0.041666666666666664          |
|ba78997921bbcdc

# Análise Exploratória e Limpeza

In [23]:
pagamentos_pedido_preprocessados.corr('valor_pagamento_normalizado','parcelas_pagamento_normalizado') # Calcula o coeficiente de correlação entre as colunas normalizadas de pagamento e parcelas. Este é um passo de Análise Exploratória de Dados (EDA)

0.33081084451898507

In [24]:
itens_pedido_df = join_itens_pedido.na.drop() # Remove todas as linhas do DataFrame join_itens_pedido que contêm qualquer valor nulo (na.drop()), criando o DataFrame limpo itens_pedido_df

# Define que as colunas de peso e dimensões do produto serão agrupadas em um único vetor chamado caracteristicas
montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g','comprimento_produto_cm','altura_produto_cm','largura_produto_cm'],
    outputCol='caracteristicas'
)

itens_pedido_df = montar_vetor.transform(itens_pedido_df) # Aplica o VectorAssembler ao DataFrame
itens_pedido_df.show()

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------+--------------------+---------------

In [25]:
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado') # Define o transformador que será usado para normalizar o vetor caracteristicas para um intervalo de 0 a 1
modelo_scaler = scaler.fit(itens_pedido_df) # O fit calcula os valores mínimos e máximos do vetor caracteristicas

itens_pedido_df = modelo_scaler.transform(itens_pedido_df) # Aplica a normalização, gerando o vetor caracteristicas_normalizado

# Treinamento e Avaliação do Modelo

In [26]:
train_data, test_data = itens_pedido_df.randomSplit([0.7, 0.3], seed=42) # Divide o DataFrame limpo em conjuntos de treinamento (70%) e teste (30%) de forma aleatória e reprodutível (seed=42)

In [27]:
#modelo de regressão linear
regressao_linear = LinearRegression(featuresCol='caracteristicas', labelCol='valor_frete') # Define o modelo de Regressão Linear. Define o vetor de features de entrada (featuresCol) e a coluna alvo a ser prevista (labelCol), que é o valor do frete
modelo_lr = regressao_linear.fit(train_data) # Treina o modelo usando o conjunto de dados de treinamento (train_data)

avaliar_test = modelo_lr.evaluate(test_data) # Avalia a performance do modelo treinado no conjunto de dados de teste (test_data)
print("RMSE (erro quadrático médio da raiz): ", avaliar_test.rootMeanSquaredError) # Imprime o RMSE (Root Mean Squared Error), que mede o erro médio das previsões.
print("r2 (coeficiente de determinação):", avaliar_test.r2) # mprime o R-squared (Coeficiente de Determinação), que mede a proporção da variância do rótulo que é explicada pelo modelo (quanto mais próximo de 1, melhor)

RMSE (erro quadrático médio da raiz):  12.292613323620843
r2 (coeficiente de determinação): 0.36567675519446763


In [28]:
spark.stop() # Encerra a SparkSession e libera os recursos